In [1]:
from ultralytics.utils.loss import v8DetectionLoss
from ultralytics import YOLO
import torch


model = YOLO('yolov8n.pt') 

results = model(['coco8/images/train/000000000034.jpg'])[0]
results


0: 448x640 1 zebra, 84.0ms
Speed: 2.6ms preprocess, 84.0ms inference, 0.9ms postprocess per image at shape (1, 3, 448, 640)


/home/dnth/anaconda3/envs/data-flywheel/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1708025831440/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant',

In [2]:
%matplotlib inline

results.show()

In [3]:
import torch

def box_iou(box1, box2):
    """
    Calculate the Intersection over Union (IoU) between two sets of bounding boxes.

    Args:
        box1 (torch.Tensor): First set of bounding boxes, shape (N, 4).
        box2 (torch.Tensor): Second set of bounding boxes, shape (M, 4).

    Returns:
        torch.Tensor: IoU values between each pair of boxes, shape (N, M).
    """
    # Compute the coordinates of the intersection rectangle
    inter_left_upper = torch.max(box1[:, None, :2], box2[:, :2])  # (N, M, 2)
    inter_right_bottom = torch.min(box1[:, None, 2:], box2[:, 2:])  # (N, M, 2)
    inter_wh = (inter_right_bottom - inter_left_upper).clamp(min=0)  # (N, M, 2)
    inter_area = inter_wh[:, :, 0] * inter_wh[:, :, 1]  # (N, M)

    # Compute the areas of the bounding boxes
    box1_area = (box1[:, 2] - box1[:, 0]) * (box1[:, 3] - box1[:, 1])  # (N,)
    box2_area = (box2[:, 2] - box2[:, 0]) * (box2[:, 3] - box2[:, 1])  # (M,)

    # Compute the union area
    union_area = box1_area[:, None] + box2_area - inter_area

    # Compute IoU
    iou = inter_area / union_area

    return iou

# Read the target annotations from the file
with open("coco8/labels/train/000000000034.txt", "r") as file:
    target_data = file.read().strip().split()

# Convert the target annotations to the required format
class_id = int(target_data[0])
x, y, w, h = map(float, target_data[1:])
target_bbox = torch.tensor([[x - w / 2, y - h / 2, x + w / 2, y + h / 2]], dtype=torch.float32)

# Extract the predicted bounding boxes from the results
pred_boxes = results.boxes.xyxy

# Move the target_bbox tensor to the same device as pred_boxes
target_bbox = target_bbox.to(pred_boxes.device)

# Calculate the IoU between predicted and target bounding boxes
iou = box_iou(pred_boxes, target_bbox)

# Find the index of the predicted box with the highest IoU
best_match_idx = iou.argmax()

# Get the IoU value for the best match
best_iou = iou[best_match_idx].item()

print("Best IoU:", best_iou)

Best IoU: 0.0


/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/dnth/snap/code/common/.cache/gio-modules/libgiolibproxy.so


eog: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_init, version GLIBC_PRIVATE
